In [ ]:
import torch
import numpy as np 

torch.cuda.is_available()

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CustomGRU(nn.Module):
    def __init__(self, bert_hidden_size, gru_hidden_size, num_classes):
        super(CustomGRU, self).__init__()

        # GRU layer
        self.gru = nn.GRU(bert_hidden_size, gru_hidden_size, batch_first=True,
                          bidirectional=False, dropout=0.3, num_layers=3)


        # Fully connected layer for classification.
        self.fc = nn.Linear(gru_hidden_size, num_classes)

  

    def forward(self, x):

        # GRU layer.
        gru_out, _ = self.gru(x)
        
        # Selecting the output from the last time step of all sequences.
        x = gru_out[:, -1, :]


        # Output layer.
        output = self.fc(x)


        return output
    

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CustomLSTM(nn.Module):
    def __init__(self, bert_hidden_size, lstm_hidden_size, num_classes):
        super(CustomLSTM, self).__init__()

        # Bidirectional LSTM layer with dropout and multiple layers.
        self.lstm = nn.LSTM(bert_hidden_size, lstm_hidden_size, batch_first=True,
                            bidirectional=True, num_layers=1)

    
        # Fully connected layer for classification.
        self.fc = nn.Linear(lstm_hidden_size*2, num_classes)

        # Dropout layer to prevent overfitting.
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):


        # LSTM layer.
        lstm_out, _ = self.lstm(x)

        # Selecting the output from the last time step of all sequences.
        x = lstm_out[:, -1, :]

     
        x = self.dropout(x)

        # Output layer.
        output = self.fc(x)



        return output

In [ ]:
class CNN_NLP(nn.Module):
 
    def __init__(self,
                 
                 vocab_size=1024,
            
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=4,
                 dropout=0.5):


        super(CNN_NLP, self).__init__()

    
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=vocab_size,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):


       
        x_embed = input_ids.float()


        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class CustomUNLSTM(nn.Module):
    def __init__(self, bert_hidden_size, lstm_hidden_size, num_classes):
        super(CustomUNLSTM, self).__init__()

        # Bidirectional LSTM layer with dropout and multiple layers.
        self.lstm = nn.LSTM(bert_hidden_size, lstm_hidden_size, batch_first=True,
                            bidirectional=False, num_layers=1)

       

        # Fully connected layer for classification.
        self.fc = nn.Linear(lstm_hidden_size, num_classes)

        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        # Reshape BERT embeddings.

        # LSTM layer.
        lstm_out, _ = self.lstm(x)
  
        # Selecting the output from the last time step of all sequences.
        x = lstm_out[:, -1, :]


        x = self.dropout(x)

        # Output layer.
        output = self.fc(x)

    

        return output

In [ ]:

class CustomBIGRU(nn.Module):
    def __init__(self, bert_hidden_size, gru_hidden_size, num_classes):
        super(CustomBIGRU, self).__init__()

        # Bidirectional LSTM layer with dropout and multiple layers.
        self.gru = nn.GRU(bert_hidden_size, gru_hidden_size, batch_first=True,
                          bidirectional=True, dropout=0.2, num_layers=3)

        # Fully connected layer for classification.
        self.fc = nn.Linear(gru_hidden_size * 2, num_classes)

        # Dropout layer to prevent overfitting.
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
       

        # GRU layer.
        gru_out, _ = self.gru(x)
        
        # Selecting the output from the last time step of all sequences.
        x = gru_out[:, -1, :]

 
        x = self.dropout(x)

        # Output layer.
        output = self.fc(x)

        return output
    
    

The program can work with any of the four datasets with minor modifications. This program is adapted for the relabeled version of the mixed-label dataset. The program is written and commented with the help of ChatGPT and Copilot.

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

norbert_dataset = load_dataset("Statistikkprosjekt/Mixed")
T5_list = []
for i in norbert_dataset["test"]:
    T5_list.append({"review": i["review"], "polarity": str(i["polarity"])})
T5_dataset = DatasetDict({"test": Dataset.from_list(T5_list)})

In [ ]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ltg/norbert3-large")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True, max_length = 90, return_tensors="pt")
def tokenize_label_function(examples):
    return tokenizer(examples["polarity"], truncation=True, max_length = 1, return_tensors="pt")

norbert_dataset = norbert_dataset.map(tokenize_function, batched=True)
T5_dataset = T5_dataset.map(tokenize_label_function, batched=True)

In [ ]:

T5_dataset = T5_dataset.rename_column("input_ids","labels")
T5_dataset = T5_dataset.remove_columns(["token_type_ids","attention_mask","polarity"])
T5_dataset = T5_dataset.map(tokenize_function, batched=True)
T5_dataset = T5_dataset.remove_columns(["review","token_type_ids"])


norbert_dataset = norbert_dataset.remove_columns(["review"])
norbert_dataset = norbert_dataset.rename_column("polarity", "labels")

In [ ]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

In [ ]:

norbert_dataset.set_format("torch")
T5_dataset.set_format("torch")


In [ ]:


norbert_dataset = norbert_dataset["test"]
T5_dataset = T5_dataset["test"]

In [ ]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ltg/norbert3-base")
from transformers import AutoModel
norbert = AutoModel.from_pretrained("ltg/norbert3-large", trust_remote_code=True)


In [ ]:
from torch.utils.data import DataLoader
batch_size = 10

norbert_dataloader = DataLoader(norbert_dataset, batch_size=batch_size)
T5_dataloader= DataLoader(T5_dataset,batch_size=batch_size)
# 
model_files = [
    #Example file names
    "norbert1.pth", "norbert2.pth", "norT5_1.pth",
    #Models that are embedded with a specific NorBERT needs to be in a list of length 2 with the NorBERT
    ["BiGRU1.pth","NBERT_BIGRU1.pth,"], ['CNN7.pth','NBERT_CNN7.pth']



               ]
len(model_files)

In [ ]:

from sklearn.metrics import precision_score
import tqdm
precision_metric = precision_score  # You can use other metrics suitable for your task

# Initialize an array to store individual model predictions
all_predictions = []
all_true_labels = []
all_logits = []

a = 0
for i in model_files:
    torch.cuda.empty_cache()
   
    print(i)
    predictions = []
    logits_list = []
    if "norbert" in i :
        model = torch.load(i)
        model.to(device)
        model.eval()
        for batch in norbert_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            
            logits = outputs.logits 
            logits_list.append(logits.cpu().numpy())
        

            # Extract true labels for the current batch
            if a == 0:
                true_labels_batch = batch["labels"].cpu().numpy()
                all_true_labels.extend(true_labels_batch)
    elif "T5" in i:
            model = torch.load(i)
            model.to(device)
            model.eval()
            for batch in T5_dataloader:
                
                batch = {k: v.to(device) for k, v in batch.items()}
                

                            
                with torch.no_grad():
                    # Make sure to provide the correct input tensors for your model
                    outputs = model(**batch)
        
 
                
                # Process the logits according to your task
                logits = outputs.logits
                
                indices = torch.tensor([920,561  ,414 , 395])
                selected_elements = logits[:, 1, indices]
                logits_list.append(selected_elements.cpu().numpy())
               
    else:
        model = torch.load(i[0])
        model.to(device)
        model.eval()
        norbert = torch.load(i[1])
        norbert.to(device)
        norbert.eval()
    
        for batch in norbert_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
           

            with torch.no_grad():
                norbert_outputs = norbert(**batch)
                
                batch_hidden_states = norbert_outputs.last_hidden_state
                outputs = model(batch_hidden_states)
              
            
            logits = outputs
            logits_list.append(logits.cpu().numpy())
        

            # Extract true labels for the current batch
            if a == 0:
                true_labels_batch = batch["labels"].cpu().numpy()
                all_true_labels.extend(true_labels_batch)
        del norbert
    

    flattened_list = [result for batch in logits_list for result in batch]
    all_logits.append(flattened_list)
    a = 1



#import winsound

# Your program code goes here

# Beep to indicate program completion
#winsound.Beep(1000, 500)  #

In [ ]:
DENNE = np.array(all_logits)
all_true_labels= np.array(all_true_labels)

In [ ]:
all_logits = DENNE

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
ac_list = []
f1_list = []
recall_list = []
precision_list = []
for i in all_logits:
    tmp = np.array(i)
    print(tmp.shape)
    

    


    bagging_predictions = np.argmax(tmp, axis=1)
    #print(bagging_predictions.shape)

    ac_list.append(accuracy_score(y_true=all_true_labels, y_pred=bagging_predictions))
    f1_list.append(f1_score(y_true=all_true_labels, y_pred=bagging_predictions,average='macro'))
    recall_list.append(recall_score(y_true=all_true_labels, y_pred=bagging_predictions,average='macro'))
    precision_list.append(precision_score(y_true=all_true_labels, y_pred=bagging_predictions,average='macro'))

In [ ]:
lists = [ f1_list,ac_list, recall_list, precision_list]
selection = 0
mean = np.mean(lists[selection])*100
std = np.std(lists[selection])*100

# Print the result in the specified format
for i in lists:
    mean = np.mean(i)*100
    std = np.std(i)*100

    print(f"${mean:.4f}^{{\pm{std:.4f}}}$ &",end="")

In [ ]:
bagging_logits = np.mean(all_logits, axis=0)

print(bagging_logits.shape)
bagging_predictions = np.argmax(bagging_logits, axis=1)
print(bagging_predictions.shape)
# Calculate F1 score
f1 = f1_score(all_true_labels, bagging_predictions, average='macro')
ac = accuracy_score(all_true_labels, bagging_predictions)
recall = recall_score(all_true_labels, bagging_predictions, average='macro')
precision = precision_score(all_true_labels, bagging_predictions, average='macro')
print(f"Test F1 Score: {f1:.3f}")
print(f"Test Accuracy: {ac:.3f}")
print(f"Test Recall: {recall:.3f}")
print(f"Test Precision: {precision:.3f}")
for i in [f1,ac,recall,precision]:
    

    print(f"${i*100:.4f}$ &",end="")

In [ ]:
import random
import numpy as np
all_logits = np.array(DENNE)
all_logits.shape
# Get the shape of all_logits
shape = all_logits.shape
i =1
f1_list = []
ac_list = []
recall_list = []
precision_list = []

# Select two random indices
for i in range(1,16):
    
    f1_list.append([])
    ac_list.append([])
    recall_list.append([])
    precision_list.append([])
    for _ in range(5000):
        indices = random.sample(range(shape[0]), i)
        # Create a new tensor with shape (2, 1264, 4)
        new_tensor = np.zeros((i, shape[1], shape[2]))

        # Assign the selected indices from all_logits to the new tensor
        for j in range(i):
            new_tensor[j] = all_logits[indices[j]]

        bagging_logits = np.mean(new_tensor, axis=0)
        bagging_predictions = np.argmax(bagging_logits, axis=1)
   
        f1 = f1_score(all_true_labels, bagging_predictions, average='macro')
        ac = accuracy_score(all_true_labels, bagging_predictions)
        recall = recall_score(all_true_labels, bagging_predictions, average='macro')
        precision = precision_score(all_true_labels, bagging_predictions, average='macro')
        f1_list[i-1].append(f1)
        ac_list[i-1].append(ac)
        recall_list[i-1].append(recall)
        precision_list[i-1].append(precision)



In [ ]:
for i in [f1_list[14],ac_list[14],recall_list[14],precision_list[14]]:
    mean_list = []
    std_list = []
    mean = np.mean(i)*100
    std = np.std(i)*100
    mean_list.append(mean)
    std_list.append(std)
    print(f"${mean:.4f}^{{\pm{std:.4f}}}$ &",end="")





In [ ]:
for i in [f1_list[4],ac_list[4],recall_list[4],precision_list[4]]:
    mean_list = []
    std_list = []
    mean = np.mean(i)*100
    std = np.std(i)*100
    mean_list.append(mean)
    std_list.append(std)
    print(f"${mean:.4f}^{{\pm{std:.4f}}}$ &",end="")

In [ ]:
import plotly.graph_objects as go

import plotly.io as pio
#io.renderers.default = 'png'
mean_list = np.mean(f1_list, axis=1)
import plotly.express as px
import pandas as pd
df = pd.DataFrame(mean_list, columns = ['f1'])
df['Number Of Models In Ensemble'] = range(1,16)
plt = px.line(df, x='Number Of Models In Ensemble', y='f1', title='F1 Macro Score Across Ensemble Sizes on Test Set of Mixed-Label Dataset',
              labels={"epoch": "Epoch", "f1": "F1 Score"}, 
              template='plotly')

plt.update_traces(line=dict(width=2.5, color='darkred'), 
                  mode='lines', 
                  marker=dict(size=8, color='LightSkyBlue', line=dict(width=2, color='DarkSlateGrey')))
plt.update_layout(title_font_size=24, title_x=0.5, 
                  xaxis_title_font=dict(size=18), yaxis_title_font=dict(size=18),
                  xaxis_gridcolor='gray', yaxis_gridcolor='gray')
#plt.to_image(format="png", width=600, height=350, scale=2)
plt.show()


In [ ]:
mean_list = np.mean(ac_list, axis=1)
import plotly.express as px
import pandas as pd
df = pd.DataFrame(mean_list, columns = ['f1'])
df['Number Of Models In Ensemble'] = range(1,16)
plt = px.line(df, x='Number Of Models In Ensemble', y='f1', title='Accuracy Across Ensemble Sizes on Test Set of Mixed-Label Dataset',
              labels={"epoch": "Epoch", "f1": "Accuracy"}, 
              template='plotly_white')

plt.update_traces(line=dict(width=2.5, color='darkred'), 
                  mode='lines', 
                  marker=dict(size=8, color='LightSkyBlue', line=dict(width=2, color='DarkSlateGrey')))
plt.update_layout(title_font_size=24, title_x=0.5, 
                  xaxis_title_font=dict(size=18), yaxis_title_font=dict(size=18),
                  xaxis_gridcolor='gray', yaxis_gridcolor='gray')

plt.show()

In [ ]:
mean_list = np.mean(recall_list, axis=1)
import plotly.express as px
import pandas as pd
df = pd.DataFrame(mean_list, columns = ['f1'])
df['Number Of Models In Ensemble'] = range(1,16)
plt = px.line(df, x='Number Of Models In Ensemble', y='f1', title='Recall Macro Score Across Ensemble Sizes on Test Set of Mixed-Label Dataset',
              labels={"epoch": "Epoch", "f1": "Recall"}, 
              template='plotly_white')

plt.update_traces(line=dict(width=2.5, color='darkred'), 
                  mode='lines', 
                  marker=dict(size=8, color='LightSkyBlue', line=dict(width=2, color='DarkSlateGrey')))
plt.update_layout(title_font_size=24, title_x=0.5, 
                  xaxis_title_font=dict(size=18), yaxis_title_font=dict(size=18),
                  xaxis_gridcolor='gray', yaxis_gridcolor='gray')

plt.show()

In [ ]:
mean_list = np.mean(precision_list, axis=1)
import plotly.express as px
import pandas as pd
df = pd.DataFrame(mean_list, columns = ['f1'])
df['Number Of Models In Ensemble'] = range(1,16)
plt = px.line(df, x='Number Of Models In Ensemble', y='f1', title='Precision Macro Score Across Ensemble Sizes on Test Set of Mixed-Label Dataset',
              labels={"epoch": "Epoch", "f1": "Precision"}, 
              template='plotly_white')

plt.update_traces(line=dict(width=2.5, color='darkred'), 
                  mode='lines', 
                  marker=dict(size=8, color='LightSkyBlue', line=dict(width=2, color='DarkSlateGrey')))
plt.update_layout(title_font_size=24, title_x=0.5, 
                  xaxis_title_font=dict(size=18), yaxis_title_font=dict(size=18),
                  xaxis_gridcolor='gray', yaxis_gridcolor='gray')
#plt.to_image(format="png", width=600, height=350, scale=2)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# Calculate confusion matrix
conf_matrix = confusion_matrix(all_true_labels, bagging_predictions)

# Display the confusion matrix
plt.figure(figsize=(10, 8))
c = sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Neutral","Positive","Negative", "mixed"], yticklabels=["Neutral","Positive","Negative", "mixed"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
c.collections[0].colorbar.remove()
plt.ylabel("True Label")
plt.show() 


In [ ]:


# Calculate percentages for each true label
total_true_labels = np.sum(conf_matrix, axis=1)
percentages = (conf_matrix / total_true_labels[:, np.newaxis]) * 100

# Replace NaN values with 0 (for cases where the true label count is 0)
percentages = np.nan_to_num(percentages)

In [ ]:
plt.figure(figsize=(10, 8))
c = sns.heatmap(percentages, annot=True, fmt="f", cmap="Blues", xticklabels=["Neutral","Positive","Negative", "mixed"], yticklabels=["Neutral","Positive","Negative", "mixed"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
c.collections[0].colorbar.remove()
plt.ylabel("True Label")
plt.show() 

